# CDCTC Single Parent Work Requirement Reform Analysis

This notebook analyzes a reform to the Child and Dependent Care Tax Credit (CDCTC) that would make families eligible if **at least one parent works**, instead of the current law requiring **both parents to work**.

## Background

Under current law, the CDCTC caps eligible childcare expenses at the **minimum** of both spouses' earnings for married couples. This means if one spouse earns $50,000 and the other earns $0, the cap is $0 and the family receives no credit.

The reform changes this to use the **maximum** of both spouses' earnings, allowing families to qualify if at least one parent works.

In [ ]:
from policyengine_us import Microsimulation
from policyengine_us.reforms.cdcc.cdcc_single_parent_work_requirement import (
    cdcc_single_parent_work_requirement
)
import pandas as pd

## Microsimulation Analysis (2026)

In [ ]:
YEAR = 2026

# Baseline simulation
print("Running baseline simulation...")
baseline = Microsimulation()

# Reform simulation
print("Running reform simulation...")
reform = Microsimulation(reform=cdcc_single_parent_work_requirement)

# Clear cached downstream variables to ensure recalculation
# Note: This reform overrides cdcc_relevant_expenses directly 
for var_name in ['cdcc_relevant_expenses', 'cdcc_potential', 'cdcc', 'capped_cdcc', 'income_tax']:
    holder = reform.get_holder(var_name)
    if holder:
        holder.delete_arrays()

# Calculate federal impacts using income_tax (isolates federal budgetary impact)
baseline_income_tax = baseline.calculate('income_tax', YEAR)
reform_income_tax = reform.calculate('income_tax', YEAR)
federal_revenue_change = reform_income_tax.sum() - baseline_income_tax.sum()

# Also get CDCC-specific metrics
baseline_cdcc = baseline.calculate('cdcc', YEAR)
reform_cdcc = reform.calculate('cdcc', YEAR)

print("Done.")

In [ ]:
# Calculate metrics
cdcc_cost = reform_cdcc.sum() - baseline_cdcc.sum()

baseline_claimants = (baseline_cdcc > 0).sum()
reform_claimants = (reform_cdcc > 0).sum()
new_claimants = reform_claimants - baseline_claimants

baseline_avg = baseline_cdcc[baseline_cdcc > 0].mean()
reform_avg = reform_cdcc[reform_cdcc > 0].mean()

# New claimants average
new_claimant_mask = (baseline_cdcc.values == 0) & (reform_cdcc.values > 0)
new_claimants_avg = reform_cdcc[new_claimant_mask].mean() if new_claimant_mask.any() else 0

print(f"Federal Budgetary Impact (income_tax change): ${federal_revenue_change/1e9:.2f}B")
print(f"CDCC Credit Change: ${cdcc_cost/1e9:.2f}B")

In [ ]:
# Create results table
results = pd.DataFrame({
    'Metric': ['CDCC Cost', 'Income Tax Revenue', 'Claimants', 'Average Credit'],
    'Baseline': [
        f'${baseline_cdcc.sum()/1e9:.2f}B',
        f'${baseline_income_tax.sum()/1e9:.2f}B',
        f'{baseline_claimants/1e6:.2f}M',
        f'${baseline_avg:.0f}',
    ],
    'Reform': [
        f'${reform_cdcc.sum()/1e9:.2f}B',
        f'${reform_income_tax.sum()/1e9:.2f}B',
        f'{reform_claimants/1e6:.2f}M',
        f'${reform_avg:.0f}',
    ],
    'Change': [
        f'+${cdcc_cost/1e9:.2f}B',
        f'${federal_revenue_change/1e9:.2f}B',
        f'+{new_claimants/1e6:.2f}M',
        f'${reform_avg - baseline_avg:+.0f}',
    ]
})

results

## Summary

The CDCTC single parent work requirement reform would:

- **Federal CDCC cost increase**: ~$740M annually
- **Federal income tax revenue change**: ~-$437M (less than CDCC increase because CDCC is non-refundable)
- **State CDCC impact**: ~$198M additional state credits
- **New claimants**: ~920,000 additional families

The reform primarily benefits one-earner married couples with childcare expenses who are currently excluded from the credit.

**Implementation note:** This reform overrides `cdcc_relevant_expenses` directly rather than `min_head_spouse_earned` to avoid affecting other programs that use the minimum earnings calculation for non-CDCC purposes.

## State CDCC Impacts

Some states offer child care credits that directly match or piggyback on the federal CDCC. These state credits will also increase under the reform since they reference the federal credit amount.

In [ ]:
# Use aggregated state_cdcc for total state impact
baseline_state_cdcc = baseline.calculate('state_cdcc', YEAR)
reform_state_cdcc = reform.calculate('state_cdcc', YEAR)
total_state_change = reform_state_cdcc.sum() - baseline_state_cdcc.sum()

# For state-by-state breakdown, calculate key state CDCCs directly
# (Entity mapping between tax_unit and household makes aggregation complex)
state_vars = [
    ('California', 'ca_cdcc'),
    ('South Carolina', 'sc_cdcc'),
    ('Pennsylvania', 'pa_cdcc'),
    ('Maryland', 'md_cdcc'),
    ('DC', 'dc_cdcc'),
    ('Rhode Island', 'ri_cdcc'),
    ('Kansas', 'ks_cdcc'),
    ('Arkansas', 'ar_cdcc'),
    ('Delaware', 'de_cdcc'),
    ('West Virginia', 'wv_cdcc'),
    ('Nebraska', 'ne_cdcc_nonrefundable'),
]

state_results = []
for state_name, var_name in state_vars:
    try:
        b = baseline.calculate(var_name, YEAR).sum()
        r = reform.calculate(var_name, YEAR).sum()
        change = r - b
        if abs(change) > 1000:  # Only show meaningful changes
            state_results.append({
                'State': state_name,
                'Baseline ($M)': round(b/1e6, 2),
                'Reform ($M)': round(r/1e6, 2),
                'Change ($M)': round(change/1e6, 2),
            })
    except:
        pass

state_df = pd.DataFrame(state_results).sort_values('Change ($M)', ascending=False)
state_df

In [ ]:
print(f"=== Summary ===")
print(f"Federal Budgetary Impact (income_tax): ${federal_revenue_change/1e6:.0f}M")
print(f"Total State CDCC Impact (state_cdcc): ${total_state_change/1e6:.0f}M")
print(f"Combined Impact: ${(federal_revenue_change + total_state_change)/1e6:.0f}M")

## Single Household Example

In [ ]:
from policyengine_us import Simulation

# Example: Married couple where only one spouse works
situation = {
    'people': {
        'adult1': {'age': {YEAR: 35}, 'employment_income': {YEAR: 50_000}},
        'adult2': {'age': {YEAR: 35}, 'employment_income': {YEAR: 0}},
        'child': {'age': {YEAR: 5}},
    },
    'tax_units': {
        'tax_unit': {
            'members': ['adult1', 'adult2', 'child'],
            'tax_unit_childcare_expenses': {YEAR: 5_000},
        }
    },
    'families': {'family': {'members': ['adult1', 'adult2', 'child']}},
    'households': {
        'household': {
            'members': ['adult1', 'adult2', 'child'],
            'state_code': {YEAR: 'CA'},
        }
    },
    'marital_units': {
        'marital_unit': {'members': ['adult1', 'adult2']},
    },
    'spm_units': {
        'spm_unit': {'members': ['adult1', 'adult2', 'child']},
    },
}

# Baseline
baseline_sim = Simulation(situation=situation)
baseline_credit = baseline_sim.calculate('cdcc', YEAR)[0]

# Reform
reform_sim = Simulation(situation=situation, reform=cdcc_single_parent_work_requirement)
reform_credit = reform_sim.calculate('cdcc', YEAR)[0]

print(f"Example: Married couple, one earner ($50k), one child, $5k childcare expenses")
print(f"  Baseline CDCC: ${baseline_credit:.0f}")
print(f"  Reform CDCC:   ${reform_credit:.0f}")
print(f"  Benefit:       ${reform_credit - baseline_credit:.0f}")

## State Household Example: South Carolina

South Carolina offers a state CDCC equal to 7% of qualified childcare expenses (using the same expense calculation as the federal credit). Here's how the reform affects a family in South Carolina.

In [ ]:
# South Carolina example: Same family structure
sc_situation = {
    'people': {
        'adult1': {'age': {YEAR: 35}, 'employment_income': {YEAR: 50_000}},
        'adult2': {'age': {YEAR: 35}, 'employment_income': {YEAR: 0}},
        'child': {'age': {YEAR: 5}},
    },
    'tax_units': {
        'tax_unit': {
            'members': ['adult1', 'adult2', 'child'],
            'tax_unit_childcare_expenses': {YEAR: 5_000},
        }
    },
    'families': {'family': {'members': ['adult1', 'adult2', 'child']}},
    'households': {
        'household': {
            'members': ['adult1', 'adult2', 'child'],
            'state_code': {YEAR: 'SC'},
        }
    },
    'marital_units': {
        'marital_unit': {'members': ['adult1', 'adult2']},
    },
    'spm_units': {
        'spm_unit': {'members': ['adult1', 'adult2', 'child']},
    },
}

# Baseline
sc_baseline = Simulation(situation=sc_situation)
sc_baseline_federal = sc_baseline.calculate('cdcc', YEAR)[0]
sc_baseline_state = sc_baseline.calculate('sc_cdcc', YEAR)[0]

# Reform
sc_reform = Simulation(situation=sc_situation, reform=cdcc_single_parent_work_requirement)
sc_reform_federal = sc_reform.calculate('cdcc', YEAR)[0]
sc_reform_state = sc_reform.calculate('sc_cdcc', YEAR)[0]

print(f"South Carolina Example: Married couple, one earner ($50k), one child, $5k childcare")
print(f"\nFederal CDCC:")
print(f"  Baseline: ${sc_baseline_federal:.0f}")
print(f"  Reform:   ${sc_reform_federal:.0f}")
print(f"  Benefit:  ${sc_reform_federal - sc_baseline_federal:.0f}")
print(f"\nSouth Carolina CDCC (7% of federal):")
print(f"  Baseline: ${sc_baseline_state:.0f}")
print(f"  Reform:   ${sc_reform_state:.0f}")
print(f"  Benefit:  ${sc_reform_state - sc_baseline_state:.0f}")
print(f"\nTotal Combined Benefit: ${(sc_reform_federal - sc_baseline_federal) + (sc_reform_state - sc_baseline_state):.0f}")